In [1]:
import requests
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import random
import os
import datetime

In [2]:
stock ={"陽明":"2609","長榮":"2603","萬海":"2615","長榮航":"2618","遠雄港":"5607","華航":"2610","台驊投控":"2636","裕民":"2606",
        "益航":"2601" ,"東森":"2614","新興":"2605" , "四維航":"5608" , "台航":"2617" }

In [3]:
day_now =datetime.datetime.strftime(datetime.datetime.today() ,'%Y%m%d') #現在的月份
#建立url需要的日期部分，格式為年、月、1號 >>> 20190101
date=pd.date_range(start="20170101",end=day_now ,freq="MS" ).map(lambda x : x.strftime("%Y%m%d"))
#建立dataFRAM的index
date_index = pd.date_range(start="20170101",end=day_now ,freq="MS" ).map(lambda x : x.strftime("%Y%m"))

In [4]:
#header
headers ={"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}

In [5]:
def get_text(url   ,headers ):
    """負責request的請求
    傳入網址、header(避免反爬蟲機制)"""
    r = requests.get(url, headers=headers )
    res = r.text
    soup=BeautifulSoup( res ,"lxml")
    
    return soup.table

In [6]:
def get_value_PER(data):
    """請傳入 rqquest.get(url).text"""
    
    """return 資料為一個月的數值的 PBR 股價淨值比 """
    #使用BeautifulSoup 做解析
    
    df=pd.read_html(data.prettify())[0]
    df.columns = df.columns.get_level_values(1)
    PBR_end=np.median(df["股價淨值比"].map(lambda x : float(x))) #找出當月股價淨值比的中位數

    return   PBR_end

In [7]:
# 決定要儲存的資料夾
output_dir = 'Data_股價淨值比' 

# 假如資料夾不存在就新增一個資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [8]:
for name ,code  in zip(stock,stock.values()):
    url =  [f"https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date={j}&stockNo={code}" for  j in date ]
    #建立該公司的url
    PBR_=[]
    n=0 #額外停止時間
    for url_ in url:
        n+=1
        print(url_)
        data = get_text(url_ ,headers )
        time.sleep(3)
        while data is None :
            data = get_text(url_ ,headers )
            time.sleep(3)
        PBR_end =get_value_PER(data)
        PBR_.append(PBR_end)#將該公司所有的當月最高價格傳入
        if n%15==0:#每15次停5秒
            time.sleep(5)
    """將每一個公司建一個DataFrame，並建立該公司資料的csv檔，索引用日期的部分"""
    pd.DataFrame({"股價淨值比":PBR_},index= date_index).round(2).to_csv(f"./Data_股價淨值比/{name}_{code}.csv")
    
    
        

https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20170101&stockNo=2609
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20170201&stockNo=2609
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20170301&stockNo=2609
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20170401&stockNo=2609
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20170501&stockNo=2609
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20170601&stockNo=2609
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20170701&stockNo=2609
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20170801&stockNo=2609
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20170901&stockNo=2609
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20171001&stockNo=2609
https://www.twse.com.tw/exchangeReport/BWIBBU?response=html&date=20171101&stockNo=2609
https://www.twse.com.tw/exchangeReport/BWIB